In [1]:
import numpy as np
import os
import subprocess
import tensorflow as tf
from tensorflow import lite as tfl
from tensorflow.keras import layers

/home/eduard/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/eduard/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/eduard/anaconda3/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/eduard/anaconda3/lib/python3.7/site-packages/tensorboar

In [2]:
use_gdrive = False
retrain_models = False

if use_gdrive:
    from google.colab import drive

    # Directory in Google Drive were we will save data.
    BASE_FOLDER = '/content/drive/My Drive/Colab Notebooks/tfm/'
    # Just set to True if you want to retrain the models.
    retrain_models = False

    drive.mount('/content/drive/')
else:
    BASE_FOLDER = './'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device {} not found'.format(device_name))
else:
  print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [16]:
# Train LYTNet v2
path = os.path.abspath(os.getcwd()) + "/ImVisible/Model/training.py"
#exec(open(path).read())
subprocess.run(path, stdout=subprocess.PIPE).stdout

b''

In [1]:
# Convolutional layers.
def conv_layer(x):
    x = layers.Conv2D(16, (3, 3), use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    #x = layers.Activation('relu')(x)
    x = layers.Activation(tfl.hard_swish)(x)
    return x

def bottleneck_layer(x):
    

# Defining the model.
def create_model(height, width, depth):
    inputShape = (height, width, depth)
    x = layers.Input(shape=input_shape)
    x = conv_layer(x)
    x = layers.MaxPooling2D()(x)
    x = bottleneck_layer(x)
    
    

In [ ]:
###################################################################################################
# Ejemplo de como cargar de archivos con referencia al directorio base
###################################################################################################

# DESCARGAR Y DESCOMPRIMIR EL DATASET CALTECH_PREPROCESADO DEL GDRIVE
# Cargar las ndarrays
with open(BASE_FOLDER+"pickle_all_images_df.pickle", "rb") as input_file:
    x_data = pickle.load(input_file)
with open(BASE_FOLDER+"pickle_all_classes.pickle", "rb") as input_file:
    y_data = pickle.load(input_file)

In [ ]:
print(x_data.shape)
print(len(y_data))

In [ ]:
# show image
imgplot = plt.imshow(x_data[129])
plt.show()

In [ ]:
import random

# Shuffle data.
aggregated_data = list(zip(x_data, y_data))
random.shuffle(aggregated_data)
x_data, y_data = zip(*aggregated_data)
x_data = np.array(x_data)
y_data = np.array(y_data)

In [ ]:
# Transformando las labels de texto a valores numéricos
y = np.unique(y_data)
mapping = { key : value for key,value in zip(y,range(len(y)))}
processed_y = np.array([mapping[i] for i in y_data])

In [ ]:
number_of_classes = len(y)

# Split dataset, 90% train and 10% test.
split_1 = int(0.8 * len(y_data))
split_2 = int(0.9 * len(y_data))

y_train = processed_y[:split_1]
x_train = x_data[:split_1]
y_validation = processed_y[split_1:split_2]
x_validation = x_data[split_1:split_2]
y_test = processed_y[split_2:]
x_test = x_data[split_2:]

In [ ]:
import tensorflow as tf
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.models import load_model

def add_top_model(base_model, print_summary=False):
  model = models.Sequential()
  model.add(model_base)
  model.add(layers.Flatten())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dense(number_of_classes, activation='softmax'))
  if print_summary:
    model.summary()
  return model

# Compile the model.
def compile_model(model):
  model.compile(optimizer='rmsprop',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
# Data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30, # grados de rotacion aleatoria
    width_shift_range=0.2, # fraccion del total (1) para mover la imagen
    height_shift_range=0.2, # fraccion del total (1) para mover la imagen
    shear_range=0.15, # deslizamiento
    zoom_range=0.15, # rango de zoom
    horizontal_flip=True, # girar las imagenes horizontalmente (eje vertical)
    fill_mode='nearest', # como rellenar posibles nuevos pixeles
    channel_shift_range=0.1 # cambios aleatorios en los canales de la imagen
)

# Try another one.
datagen2 = ImageDataGenerator(
    rotation_range=20, # grados de rotacion aleatoria
    width_shift_range=0.2, # fraccion del total (1) para mover la imagen
    height_shift_range=0.2, # fraccion del total (1) para mover la imagen
    shear_range=0.15, # deslizamiento
    zoom_range=0.15, # rango de zoom
    horizontal_flip=True, # girar las imagenes horizontalmente (eje vertical)
    fill_mode='nearest', # como rellenar posibles nuevos pixeles
    channel_shift_range=0.2 # cambios aleatorios en los canales de la imagen
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Train the model.
def fit_model(model, datagen, x_train, y_train, x_validation, y_validation, epochs=20, augmentation=10, callbacks=None, print_plot=True):
  batch_size = 32
  num_samples = x_train.shape[0] * augmentation
  history_pretrained = model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                                           epochs=epochs,
                                           validation_data=(x_validation, y_validation),
                                           use_multiprocessing=True,
                                           workers=4,
                                           callbacks=callbacks,
                                           steps_per_epoch=num_samples//batch_size)

  if print_plot:
    # Print accuracy plot
    epochs = np.arange(1,len(history_pretrained.history['accuracy'])+1)
    plt.title('Pre trained model')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.plot(epochs,history_pretrained.history['accuracy'],'r',label='training accuracy')
    plt.plot(epochs,history_pretrained.history['val_accuracy'],'b',label='validation accuracy')
    plt.legend(loc='best')
    plt.show()


In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import resnet
from tensorflow.keras.applications import resnet_v2
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications import densenet
from tensorflow.keras.applications import nasnet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

# Testing all Keras Applications, to see which ones give better results with our set, and then try to get better results from those.
# I excluded NASNetLarge as it just accepts an input of (331, 331, 3) there is an issue about this: https://github.com/keras-team/keras-applications/issues/78
# Also same problem with NASNetMobile whitch just accepts an input of (224, 224, 3).
pre_trained_models = [Xception, VGG16, VGG19,
                      resnet.ResNet50, resnet.ResNet101, resnet.ResNet152,
                      resnet_v2.ResNet50V2, resnet_v2.ResNet101V2, resnet_v2.ResNet152V2,
                      InceptionV3, InceptionResNetV2, MobileNet,
                      densenet.DenseNet121, densenet.DenseNet169, densenet.DenseNet201,
                      #nasnet.NASNetLarge, nasnet.NASNetMobile,
                      MobileNetV2]

for pre_trained_model in pre_trained_models:
  model_base = pre_trained_model(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
  print('Pre trained model: ', model_base.name)
  if retrain_models or not os.path.isfile(BASE_FOLDER+model_base.name+'.h5'):
    model_base.trainable = False
    model = add_top_model(model_base, False)
    compile_model(model)
    fit_model(model, datagen, x_train, y_train, x_validation, y_validation, 5, 5);
    model.save(BASE_FOLDER+model_base.name+'.h5')
  else:
    model = load_model(BASE_FOLDER+model_base.name+'.h5')
  # Let's evaluate it.
  loss, accuracy = model.evaluate(x_test, y_test)
  print('loss {} accuracy {}'.format(loss, accuracy))


In [ ]:
# Looks like the best results came from VGG16 and VGG19, we can not really be sure with that low number of epochs, but I'll follow my gut.
# With more time and more processing power, I would test well every one of them.
from tensorflow.keras.applications.vgg16 import VGG16

model_name = 'trained_vgg16'
if retrain_models or not os.path.isfile(BASE_FOLDER+model_name+'.h5'):
  # Define model using VGG16 to transfer learning.
  model_base = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
  model_base.trainable = False
  # Add a top model to the base_model.
  model = add_top_model(model_base, False)

  # Compile the model.
  compile_model(model)

  # Save checkpoints to get the best weights from all epochs, same name for all, to keep the best checkpoint.
  filename = BASE_FOLDER+model_name+'checkpoint.hdf5'
  checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  callbacks = [checkpoint]

  # Fit the model.
  fit_model(model, datagen, x_train, y_train, x_validation, y_validation, 20, 10, callbacks);

  # Load best weights.
  model.load_weights(filename)

  # Compile the model with the best weights.
  compile_model(model)

  # Save the model.
  model.save(BASE_FOLDER+model_name+'.h5')
else:
  # Load the model.
  model = load_model(BASE_FOLDER+model_name+'.h5')

# Let's evaluate it.
loss, accuracy = model.evaluate(x_test, y_test)
print('loss {} accuracy {}'.format(loss, accuracy))

# Gives our best results.

In [ ]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions

# Show image
img_index = 2
img = x_data[img_index]
imgplot = plt.imshow(img)
plt.show()

# Make a prediction.
predictions = model.predict(np.array([img]))
prediction = np.argmax(predictions)

print('Predicted: ', y[prediction])
print('Real: ', y[processed_y[img_index]])


In [ ]:
# Now learning from scratch.
# I been trying a fiew models, and the following 2 where the ones to give better results.

model_name = 'custom_model_1'
if retrain_models or not os.path.isfile(BASE_FOLDER+model_name+'.h5'):
  # Define model.
  model = models.Sequential()
  # The input has 128 for 128 pixels and 3 channels, we stay that.
  # The number of channels for each convolutional layer has been chosen using trial and error, starting with a number higher than 32 or lower than 16 was giving bad results
  model.add(layers.Conv2D(32,(3,3),input_shape=(128,128,3),activation='relu'))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  # We use MaxPooling to reduce dimensionality, we group each 4 pixels and use tha max.
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  # Increase the numeber of channels slowly while we reduce dimensionality.
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  # End of the base model.
  # Start of the top model.
  # We flaten to one dimension.
  model.add(layers.Flatten())
  # One Dense of 256 was giving the best result.
  model.add(layers.Dense(256,activation='relu'))
  # Softmax activation with the number of clases we want to classify.
  model.add(layers.Dense(number_of_classes,activation='softmax'))
  #model.summary()

  # Compile the model.
  compile_model(model)

  # Save checkpoints to get the best weights from all epochs, same name for all, to keep the best checkpoint.
  filename = BASE_FOLDER+model_name+'checkpoint.hdf5'
  checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  callbacks = [checkpoint]

  # Fit the model.
  fit_model(model, datagen, x_train, y_train, x_validation, y_validation, 20, 10, callbacks);

  # Load best weights.
  model.load_weights(filename)

  # Compile the model with the best weights.
  compile_model(model)

  # Save the model.
  model.save(BASE_FOLDER+model_name+'.h5')
else:
  # Load the model.
  model = load_model(BASE_FOLDER+model_name+'.h5')

# Let's evaluate it.
loss, accuracy = model.evaluate(x_test, y_test)
print('loss {} accuracy {}'.format(loss, accuracy))


In [ ]:
# Another model learning from scratch.

model_name = 'custom_model_2'
if retrain_models or not os.path.isfile(BASE_FOLDER+model_name+'.h5'):
  # Define model.
  model = models.Sequential()
  model.add(layers.Conv2D(16,(3,3),input_shape=(128,128,3),activation='relu'))
  model.add(layers.Conv2D(16,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  model.add(layers.Conv2D(32,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.Conv2D(64,(3,3),activation='relu'))
  model.add(layers.MaxPooling2D((2,2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(256,activation='relu'))
  model.add(layers.Dense(number_of_classes,activation='softmax'))
  #model.summary()

  # Compile the model.
  compile_model(model)

  # Save checkpoints to get the best weights from all epochs, same name for all, to keep the best checkpoint.
  filename = BASE_FOLDER+model_name+'checkpoint.hdf5'
  checkpoint = ModelCheckpoint(filename, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
  callbacks = [checkpoint]

  # Fit the model.
  fit_model(model, datagen, x_train, y_train, x_validation, y_validation, 20, 10, callbacks);

  # Load best weights.
  model.load_weights(filename)

  # Compile the model with the best weights.
  compile_model(model)

  # Save the model.
  model.save(BASE_FOLDER+model_name+'.h5')
else:
  # Load the model.
  model = load_model(BASE_FOLDER+model_name+'.h5')

# Let's evaluate it.
loss, accuracy = model.evaluate(x_test, y_test)
print('loss {} accuracy {}'.format(loss, accuracy))